In [58]:
import numpy as np
from nltk import ngrams, FreqDist
import string
import re
from tqdm import tqdm
seed = 123

In [59]:
!ls data.nosync/corpora

AnnaKarenina.txt   WarAndPeace.txt    WarAndPeaceEng.txt


In [60]:
with open('data.nosync/corpora/AnnaKarenina.txt', 'r') as f:
    AK_text = f.read()
with open('data.nosync/corpora/WarAndPeace.txt', 'r') as f:
    WAP_text = f.read()

In [61]:
corpora = AK_text + WAP_text

In [85]:
def preprocess(text):
    text = re.sub(r'[^а-я\s]','', text)
    text = re.sub(r'_','', text)
    text = text.lower()
    text = ' '.join(text.split())
    return text

def gen_new_alphabet(alphabet):
    return [''.join(map(lambda x: chr(ord(x)+310), c)) for c in alphabet]

def gen_rand_map(source_alphabet, target_alphabet, seed):
    np.random.seed(seed)
    return dict(list(zip(source_alphabet,
                         np.random.permutation(target_alphabet))))

def cipher(text, mapping):
    return ''.join(mapping[c] for c in text)

def freq_decipher(text, n, corpora_freqs):
    text_freqs = FreqDist(ngrams(text, n))
    text_freqs_sorted = [''.join(a[0]) for a in
                        sorted(list(text_freqs.items()),
                                   key=lambda x: x[1], reverse=True)]
    corpora_freqs_sorted = [''.join(a[0]) for a in
                        sorted(list(corpora_freqs.items()),
                                   key=lambda x: x[1], reverse=True)]
    mapper = dict(list(zip(text_freqs_sorted, corpora_freqs_sorted)))
    return ''.join([mapper[''.join(c)] if i % n == 0 else '' for i, c in enumerate(ngrams(text, n))])

In [86]:
corpora = preprocess(corpora)

### 1. Отдельные буквы

In [87]:
freq_unigrams = FreqDist(ngrams(corpora, 1))

In [217]:
alphabet = [''.join(a) for a in sorted(list(freq_unigrams.keys()))]
print(alphabet)

[' ', 'а', 'б', 'в', 'г', 'д', 'е', 'ж', 'з', 'и', 'й', 'к', 'л', 'м', 'н', 'о', 'п', 'р', 'с', 'т', 'у', 'ф', 'х', 'ц', 'ч', 'ш', 'щ', 'ъ', 'ы', 'ь', 'э', 'ю', 'я']


In [218]:
new_alphabet = gen_new_alphabet(alphabet)
mapping = gen_rand_map(alphabet, new_alphabet, seed)
print(mapping)

{' ': 'խ', 'а': 'տ', 'б': 'ժ', 'в': 'ւ', 'г': 'պ', 'д': 'փ', 'е': 'ղ', 'ж': 'լ', 'з': 'ռ', 'и': 'ձ', 'й': 'հ', 'к': 'յ', 'л': 'թ', 'м': 'ը', 'н': 'ր', 'о': 'չ', 'п': 'շ', 'р': 'ճ', 'с': 'ս', 'т': 'ծ', 'у': 'մ', 'ф': 'ք', 'х': 'Ŗ', 'ц': 'զ', 'ч': 'ջ', 'ш': 'վ', 'щ': 'կ', 'ъ': 'ո', 'ы': 'ն', 'ь': 'ի', 'э': 'օ', 'ю': 'ց', 'я': 'է'}


In [219]:
np.random.seed(seed)
test_length = 10_000
start_idx = np.random.randint(0, len(corpora) - test_length)
test_orig = corpora[start_idx:start_idx + test_length]
test_orig[:500]

'ыла в карете она была совсем другая на была испуганная робкая пристыженная и оттого еще более прелестная на увидала его в то же мгновение как он вошел в комнату на ждала его на обрадовалась и смутилась от своей радости до такой степени что была минута именно та когда он подходил к хозяйке и опять взглянул на нее что и ей и ему и олли которая все видела казалось что она не выдержит и заплачет на покраснела побледнела опять покраснела и замерла чуть вздрагивая губами ожидая его н подошел к ней пок'

In [220]:
test_ciphered = cipher(test_orig, mapping)
test_ciphered[:500]

'նթտխւխյտճղծղխչրտխժնթտխսչւսղըխփճմպտէխրտխժնթտխձսշմպտրրտէխճչժյտէխշճձսծնլղրրտէխձխչծծչպչխղկղխժչթղղխշճղթղսծրտէխրտխմւձփտթտխղպչխւխծչխլղխըպրչւղրձղխյտյխչրխւչվղթխւխյչըրտծմխրտխլփտթտխղպչխրտխչժճտփչւտթտսիխձխսըմծձթտսիխչծխսւչղհխճտփչսծձխփչխծտյչհխսծղշղրձխջծչխժնթտխըձրմծտխձըղրրչխծտխյչպփտխչրխշչփŖչփձթխյխŖչռէհյղխձխչշէծիխւռպթէրմթխրտխրղղխջծչխձխղհխձխղըմխձխչթթձխյչծչճտէխւսղխւձփղթտխյտռտթչսիխջծչխչրտխրղխւնփղճլձծխձխռտշթտջղծխրտխշչյճտսրղթտխշչժթղփրղթտխչշէծիխշչյճտսրղթտխձխռտըղճթտխջմծիխւռփճտպձւտէխպմժտըձխչլձփտէխղպչխրխշչփչվղթխյխրղհխշչյ'

In [221]:
freq_decipher(test_ciphered, 1, freq_unigrams)[:500]

'пла в каресе оиа зпла товтеу дрмыая иа зпла нтьмыаииая розкая ьрнтспйеииая н оссоыо еэе золее ьрелетсиая иа мвндала еыо в со йе уыиовеине как ои вошел в коуиасм иа йдала еыо иа озрадовалатб н тумснлатб ос твоеж радотсн до сакож тсеьеин чсо зпла унимса нуеиио са коыда ои ьодюоднл к юогяжке н оьясб вгыляимл иа иее чсо н еж н еум н оллн косорая вте вндела кагалотб чсо оиа ие впдерйнс н гаьлачес иа ьократиела ьозледиела оьясб ьократиела н гауерла чмсб вгдраынвая ымзаун ойндая еыо и ьодошел к иеж ьок'

### 2. Все то же самое, только для биграмм

In [70]:
freq_bigrams = FreqDist(ngrams(corpora, 2))

In [71]:
freq_decipher(test_ciphered, 2, freq_bigrams)[:500]

'ев вергоакри онов ев всьоеойорныба пнов ев вднквба эазскатго помдндьс  эазна очнт о зми ымпри омзаолшааз с вмыий т внно ертотни юсстчелаи го ум  нцеза н китноув с всм т внно но омабея  т ркоа бллын  рковае каемскбеесилоро х поовь ожена асо об богвелы втиенстй  впоеп вм  итеис жл  уисезеяи а ьнжнкоытыврясв с внеи асо а емнавоидна орма потоот поеи  дхо б кос тескоасо м  вне ньз гумю а  чыбоюмо с вонгл рне б иатпрсаза вьнжнкоонгл рне бнавсму г бделыкоытедулзназимрьмуа диийазвоал с итецеза к сем иад'

Стало, на мой взгляд, еще хуже. Вероятно выбранный фрагмент текста слишком маленький для того, чтобы получить похожее распределение биграмм.

### 3. Monte Carlo Markov Chain sampling

Идея в том, чтобы сэмлить перестановки и оценивать прадоподобия этих перестановок. Наиболее правдоподобные будет иметь большую плотность и будут чаще сэмплироваться. В качестве функции правдоподобия будем считать 
$$\Pi_{g \in ngrams} freq(g)^{freq'(g)}$$ 
А точнее его логарифм:
$$\Sigma_{g \in ngrams} freq'(g)log(freq(g))$$
Где $freq(g)$ частота n-граммы $g$ в большом корпусе, а $freq'(g)$ частота в данном тексте.

In [140]:
def decipher(text, mapping):
    return ''.join(mapping[c] for c in text)

def calc_log_density(text, mapping, n, true_ngram_freqs):
    text = cipher(text, mapping)
    text_freqs = FreqDist(ngrams(text, n))
    total_log_density = 0
    for ngram, freq in text_freqs.items():
        if true_ngram_freqs[ngram] == 0:
            continue
        total_log_density += freq * np.log(true_ngram_freqs[ngram])
    return total_log_density

def gen_new_mapping(mapping):
    mapping = mapping.copy()
    for _ in range(1):
        x, y = np.random.choice(list(mapping.keys()), 2, replace=True)
        tmp = mapping[x]
        mapping[x] = mapping[y]
        mapping[y] = tmp
    return mapping

In [155]:
def metropolis_hastings_sample(text, mapping, n_iter, n, true_ngram_freqs, alphabet):

    for i in range(n_iter):
        # sample 
        new_mapping = gen_new_mapping(mapping)
        
        log_p_x_prime = calc_log_density(text, new_mapping, 2, true_ngram_freqs)
        log_p_x_t = calc_log_density(text, mapping, 2, true_ngram_freqs)
        
        a = min(1, np.exp(log_p_x_prime - log_p_x_t))
        
        if np.random.uniform() < a and i < n_iter-1:
            mapping = new_mapping
            
    return mapping, log_p_x_t

In [225]:
def sampling(ciphered_text, n, ngrams_freqs, alphabet, n_tries, n_iters, first_n_iters, rnd_seed=None, cipher_alphabet=None):
    if rnd_seed is None:
        rnd_seed = np.random.randint(0, 1000)
    print(f'Random seed: {rnd_seed}')
    if cipher_alphabet:
        random_inv_mapping = gen_rand_map(cipher_alphabet, alphabet, rnd_seed)
    else:
        random_inv_mapping = gen_rand_map(gen_new_alphabet(alphabet), alphabet, rnd_seed)
    inv_mapping, best_density = metropolis_hastings_sample(ciphered_text, random_inv_mapping,
                                         first_n_iters, n, ngrams_freqs, alphabet)
    best_inv_mapping = inv_mapping
    for _ in tqdm(range(n_tries)):
        inv_mapping, log_density = metropolis_hastings_sample(ciphered_text, inv_mapping,
                                         n_iters, n, ngrams_freqs, alphabet)
        if log_density > best_density:
            best_density = log_density
            best_inv_mapping = inv_mapping
    return best_inv_mapping

In [161]:
inv_mapping = sampling(test_ciphered, 2, freq_bigrams, alphabet, 100, 50, 300, rnd_seed=170)
cipher(test_ciphered, inv_mapping)[:500]

<ipython-input-155-6e19c05c6c0d>:10: RuntimeWarning: overflow encountered in exp
  a = min(1, np.exp(log_p_x_prime - log_p_x_t))
100%|██████████| 100/100 [01:51<00:00,  1.11s/it]


'ыла в карете она была совсем другая на была испуганная робкая пристыженная и оттого еще более прелестная на увидала его в то же мгновение как он вошел в комнату на ждала его на обрадовалась и смутилась от своей радости до такой степени что была минута именно та когда он подходил к хозяйке и опять взглянул на нее что и ей и ему и олли которая все видела казалось что она не выдержит и заплачет на покраснела побледнела опять покраснела и замерла чуть вздрагивая губами ожидая его н подошел к ней пок'

In [174]:
ciphered_message = 'დჳჵჂႨშႼႨშჂხჂჲდႨსႹႭჾႣჵისႼჰႨჂჵჂႨႲႹႧჲჂႨსႹႭჾႣჵისႼჰႨჲდႩჳჲႨჇႨႠჲႹქႹႨჳႹႹჱჶდსჂႽႨႩႹჲႹႭႼჰႨჵდქႩႹႨႲႭႹႧჂჲႣჲიႨჳႩႹႭდდႨშჳდქႹႨშႼႨშჳდႨჳხდჵႣჵჂႨႲႭႣშჂჵისႹႨჂႨႲႹჵჇႧჂჲდႨჾႣႩჳჂჾႣჵისႼჰႨჱႣჵჵႨეႣႨႲႹჳჵდხსდდႨႧდჲშდႭჲႹდႨეႣხႣსჂდႨႩჇႭჳႣႨႾႹჲႽႨႩႹსდႧსႹႨႽႨსჂႧდქႹႨსდႨႹჱდჶႣნ'

In [175]:
print(len(set(ciphered_message)), len(alphabet))

28 33


Чтобы переиспользовать уже написанный код, я искуственно дополню новый алфавит, чтобы длина алфавитов была равна

In [180]:
cipher_alphabet = list(set(ciphered_message).union(set('ABCDE')))
len(cipher_alphabet)

33

In [197]:
inv_mapping = sampling(ciphered_message, 2, freq_bigrams, alphabet, 100,
                       100, 300, cipher_alphabet=cipher_alphabet, rnd_seed=68)
cipher(ciphered_message, inv_mapping)

  2%|▏         | 2/100 [00:00<00:14,  6.73it/s]

68


100%|██████████| 100/100 [00:10<00:00,  9.53it/s]


'если вы вимите нордальный или почти нордальный текст у этого сообжения который легко прочитать скорее всего вы все смелали правильно и получите даксидальный балл за послемнее четвертое замание курса хотя конечно я ничего не обежаю'

### Бонус

In [222]:
freq_trigrams = FreqDist(ngrams(corpora, 3))

In [224]:
inv_mapping = sampling(test_ciphered, 3, freq_trigrams, alphabet, 100, 100, 300)
cipher(test_ciphered, inv_mapping)[:500]

596


100%|██████████| 100/100 [03:28<00:00,  2.08s/it]


'рдшисицшз э имкшижрдшиомсо хиъзщяшфикшижрдшиюотщяшккшфизмжцшфитзюоэрь ккшфиюимээмями у ижмд  итз д оэкшфикшищсюъшдши ямисиэмиь ихякмс кю ицшцимкисмг дисицмхкшэщикшиьъшдши ямикшимжзшъмсшдшовиюиохщэюдшовимэиосм бизшъмоэюиъмиэшцмбиоэ т кюичэмижрдшихюкщэшиюх ккмиэшицмяъшимкитмънмъюдицинмафбц июимтфэвисаядфкщдикшик  ичэмиюи биюи хщиюимддюицмэмзшфисо исюъ дшицшашдмовичэмимкшик исръ зьюэиюиаштдшч эикшитмцзшок дшитмжд ък дшимтфэвитмцзшок дшиюиашх здшичщэвисаъзшяюсшфиящжшхюимьюъшфи ямикитмъмг дицик битмц'

Тяжело определить, что работает лучше, потому что сам алгоритм сходится далеко не всегда и для биграмм и для триграмм. Приходится подбирать параметры - кол-во попыток и кол-во шагов для взятия одного сэмла. Но если подобрать параметры, то оба алгортима (на биграммах и триграммах) работают хорошо